# Tutorial: Flag persistence generators

In this notebook, we show how to retrieve the vertices and edges responsible for the creation/destruction of persistent topological features in a Vietoris–Rips filtration. We use the same setup and kind of dataset as in the [basic tutorial](https://github.com/giotto-ai/giotto-ph/blob/main/examples/basic_tutorial.ipynb).

In [ ]:
# Install missing dependencies
import sys
!{sys.executable} -m pip install giotto-tda

In [ ]:
# Giotto-ph
from gph import ripser_parallel

# Import utils
import numpy as np
from gtda.homology._utils import _postprocess_diagrams

# To generate dataset
from sklearn import datasets

# Plotting
from plotly import graph_objects as go
from gtda.plotting import plot_diagram, plot_point_cloud

In [ ]:
# Make a noisy circle point cloud
data = datasets.make_circles(n_samples=100, noise=0.1, factor=0.7, random_state=42)[0]

# Plot the point cloud
plot_point_cloud(data)

The information on vertices and edges responsible for the creation/destruction of persistent topological features can be retrieved by passing `return_generators=True` to `ripser_parallel`. A new entry is added to the output dictionary, with key `"gens"` and corresponding value a tuple of length 4 organized schematically as follows:

  0. vertices creating and edges destroying *finite* $0$-dimensional features;
  1. edges creating and destroying *finite* $d$-dimensional features, $d \geq 1$;
  2. vertices creating *infinite* $0$-dimensional features;
  3. edges creating *infinite* $d$-dimensional features, $d \geq 1$.

(Vertices are encoded by their indices and edges as pairs of vertex indices.) In the case of entries 1 and 3 (higher dimensions), that information is organized by homology dimension. So, for example, calling `gens` this tuple (value in the dictionary):

  - `gens[1]` and `gens[3]` are lists containing `maxdim` 2D integer `numpy` arrays, while `gens[0]` and `gens[2]` are `numpy` arrays;
  - the edges creating and destroying finite features in dimension 1 are stored in `gens_finite_1 = gens[1][0]`;
  - `gens_finite_1` is a 2D integer `numpy` array with as many rows as there are finite features in dimension 1;
  - The `i`th finite feature in the 1-dimensional barcode is created by edge `gens_finite_1[i, :2]` and destroyed by edge `gens_finite_1[i, 2:]`.

This way of presenting persistence birth and death vertices/edges agrees with other persistent homology packages and in particular with [GUDHI](http://gudhi.gforge.inria.fr/).

In [ ]:
# Compute the persistence information
persistence_info = ripser_parallel(data, return_generators=True)
gens = persistence_info['gens']

Let us visualize the 1-dimensional generators for our point cloud, labelling them by the positional index of the corresponding feature in the persistence diagram and using blue for creation and red for destruction:

In [ ]:
# Plot the point cloud
fig = plot_point_cloud(data)

# In blue are the edges that create a finite persistent topological features in dimension 1.
# In red are the edges that destroy a finite persistent topological feature in dimension 1.
for i, edges in enumerate(gens[1][0]):
    birth_edge = edges[0:2]
    death_edge = edges[2:4]
    x0_create, y0_create = data[birth_edge[0]]
    x1_create, y1_create = data[birth_edge[1]]
    x0_destroy, y0_destroy = data[death_edge[0]]
    x1_destroy, y1_destroy  = data[death_edge[1]]  

    fig.add_shape(type='line',
                  x0=x0_create,
                  y0=y0_create,
                  x1=x1_create,
                  y1=y1_create,
                  line=dict(color='Blue'))
    fig.add_shape(type='line',
                  x0=x0_destroy,
                  y0=y0_destroy,
                  x1=x1_destroy,
                  y1=y1_destroy,
                  line=dict(color='Red'))
    fig.add_trace(
        go.Scatter(x=[0.5 * (x0_create + x1_create) + 0.05, 0.5 * (x0_destroy + x1_destroy) + 0.05],
                   y=[0.5 * (y0_create + y1_create), 0.5 * (y0_destroy + y1_destroy)],
                   text=[str(i), str(i)],
                   mode="text")
    )

fig.update_layout(showlegend=False)
fig.show()